In [1]:
import pandas as pd

In [2]:
# TODO: make a combined dataset for NYRP and NYC greenthumb

In [2]:
greenthumb = pd.read_csv('../data/NYC_Greenthumb_Community_Gardens.csv')
greenthumb.head(2)

# good to know that the dataset already has NTA name and borough

,PropID,Boro,Community Board,Council District,Garden Name,Address,Size,Jurisdiction,NeighborhoodName,Cross Streets,Latitude,Longitude,Postcode,Census Tract,BIN,BBL,NTA
0,NaN,M,M03,2.0,11 BC Serenity Garden,626 East 11th Street,0.054,DPR,East Village,Avenues B & C,40.727124,-73.978677,10009,28.0,1000000,1003930020,Lower East Side ...
1,NaN,B,B08,36.0,1100 Bergen Street Community Garden,1101 & 1105 Bergen Street,0.207,PRI,Crown Heights,Nostrand & New York Avenues,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Data cleaning 1: remove whitespace in NTA column

In [4]:
# before cleaning, tons of whitespace in name
greenthumb.NTA.unique()[:5]

array(['Lower East Side                                                            ',
       nan,
       'East Harlem South                                                          ',
       'East Harlem North                                                          ',
       'East Village                                                               '],
      dtype=object)

In [5]:
# remove trailing whitespace in NTA names
greenthumb['NTA'] = greenthumb.NTA.str.strip() 
greenthumb.NTA.unique()[:5] 

array(['Lower East Side', nan, 'East Harlem South', 'East Harlem North',
       'East Village'], dtype=object)

Data cleaning 2: replace boro column with actual names

In [7]:
# before cleaning, just initials
greenthumb.Boro.unique()

array(['M', 'B', 'X', 'Q', 'R'], dtype=object)

In [8]:
# replace boro initial with actual borough name
boroughs = {
    'M': 'Manhattan', 
    'B': 'Brooklyn',
    'Q': 'Queens',
    'X': 'Bronx',
    'R': 'Staten Island'
}

greenthumb['Boro'].replace(boroughs, inplace=True)
greenthumb.head(2)

,PropID,Boro,Community Board,Council District,Garden Name,Address,Size,Jurisdiction,NeighborhoodName,Cross Streets,Latitude,Longitude,Postcode,Census Tract,BIN,BBL,NTA
0,NaN,Manhattan,M03,2.0,11 BC Serenity Garden,626 East 11th Street,0.054,DPR,East Village,Avenues B & C,40.727124,-73.978677,10009,28.0,1000000,1003930020,Lower East Side
1,NaN,Brooklyn,B08,36.0,1100 Bergen Street Community Garden,1101 & 1105 Bergen Street,0.207,PRI,Crown Heights,Nostrand & New York Avenues,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Data cleaning 3: check if NTA column in this dataset is an accurate subset of NTA dataset

In [9]:
import geopandas as gpd
nta = gpd.read_file('../data/Neighborhood Tabulation Areas.geojson')
nta.head(2)

,ntacode,shape_area,county_fips,ntaname,shape_leng,boro_name,boro_code,geometry
0,BK88,54005018.7472,047,Borough Park,39247.2280737,Brooklyn,3,(POLYGON ((-73.97604935657381 40.6312759056467...
1,QN51,52488276.477,081,Murray Hill,33266.904811,Queens,4,(POLYGON ((-73.80379022888246 40.7756101117924...


In [10]:
nta.ntaname.unique()[:5] # check first 5 elements, no whitespace

array(['Borough Park', 'Murray Hill', 'East Elmhurst', 'Hollis',
       'Manhattanville'], dtype=object)

In [12]:
# not all NTAs in the community gardens dataset is inside the NTA dataset
garden_set = set(greenthumb.NTA.unique())
nta_set = set(nta.ntaname.unique())
garden_set.issubset(nta_set)

False

In [13]:
# find out which NTAs are in gardens but not in NTA...turns out to be relatively trivial
garden_set.difference(nta_set)

{'', nan}

In [15]:
# check how many records have nta == '' or nan
invalid_num = len(greenthumb[greenthumb.NTA == '']) + len(greenthumb[greenthumb.NTA.isnull()])
print(f"{invalid_num} data records with invalid NTAs")
print(f"that's {round(invalid_num/len(greenthumb), 2)*100}% of the dataset")

107 data records with invalid NTAs
that's 20.0% of the dataset


Data cleaning 4: fill in missing/null NTA info

In [16]:
# greenthumb to geocode
gt_geocode = greenthumb[(greenthumb.Address.notnull()) & (greenthumb.NTA.isnull())]
gt_geocode.shape

(104, 17)

In [18]:
gt_geocode.head(2)

,PropID,Boro,Community Board,Council District,Garden Name,Address,Size,Jurisdiction,NeighborhoodName,Cross Streets,Latitude,Longitude,Postcode,Census Tract,BIN,BBL,NTA
1,NaN,Brooklyn,B08,36.0,1100 Bergen Street Community Garden,1101 & 1105 Bergen Street,0.207,PRI,Crown Heights,Nostrand & New York Avenues,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Bronx,X12,15.0,211th Street Block Association.,Carlisle Place,0.182,NYRP,NaN,At E. 211th Street,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
gt_geocode['Full_Address'] = gt_geocode.Address + ", " + gt_geocode.Boro + ", NY"
gt_geocode = gt_geocode[['Boro', 'Address', 'Full_Address', 'NTA']]
gt_geocode.head()

/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Boro,Address,Full_Address,NTA
1,Brooklyn,1101 & 1105 Bergen Street,"1101 & 1105 Bergen Street, Brooklyn, NY",NaN
5,Bronx,Carlisle Place,"Carlisle Place, Bronx, NY",NaN
7,Brooklyn,New Lots Avenue,"New Lots Avenue, Brooklyn, NY",NaN
10,Brooklyn,64th Street,"64th Street, Brooklyn, NY",NaN
17,Brooklyn,762-764 Herkimer Place/13-21 Hunterfly,"762-764 Herkimer Place/13-21 Hunterfly, Brookl...",NaN


In [17]:
# geocode using Full_Address

In [21]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

locator = Nominatim(user_agent="myGeocoder")
geocode = RateLimiter(locator.geocode, min_delay_seconds=1) # limit the rate at which i'm making api requests, so i don't get a service timeout error

In [22]:
gt_geocode['Point'] = gt_geocode.Full_Address.apply(geocode).apply(lambda loc: tuple([loc.point[1], loc.point[0]]) if loc else None)
gt_geocode.head()


RateLimiter caught an error, retrying (0/2 tries). Called with (*('Riverside Park/138th Street, Manhattan, NY',), **{}).
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/usr/local/Cellar/python3/3.6.2/Framework

,Boro,Address,Full_Address,NTA,Point
1,Brooklyn,1101 & 1105 Bergen Street,"1101 & 1105 Bergen Street, Brooklyn, NY",NaN,"(-73.949439375, 40.67615225)"
5,Bronx,Carlisle Place,"Carlisle Place, Bronx, NY",NaN,"(-73.863012, 40.878598)"
7,Brooklyn,New Lots Avenue,"New Lots Avenue, Brooklyn, NY",NaN,"(-73.8992787, 40.6589611)"
10,Brooklyn,64th Street,"64th Street, Brooklyn, NY",NaN,"(-74.0231939, 40.6400567)"
17,Brooklyn,762-764 Herkimer Place/13-21 Hunterfly,"762-764 Herkimer Place/13-21 Hunterfly, Brookl...",NaN,None


In [23]:
len(gt_geocode[gt_geocode.Point.isnull()]) # still 55 addresses that cant be geocoded

55

In [24]:
# check that those successfully geocoded are within new york
success = gt_geocode[gt_geocode.Point.notnull()]
success.head()

,Boro,Address,Full_Address,NTA,Point
1,Brooklyn,1101 & 1105 Bergen Street,"1101 & 1105 Bergen Street, Brooklyn, NY",NaN,"(-73.949439375, 40.67615225)"
5,Bronx,Carlisle Place,"Carlisle Place, Bronx, NY",NaN,"(-73.863012, 40.878598)"
7,Brooklyn,New Lots Avenue,"New Lots Avenue, Brooklyn, NY",NaN,"(-73.8992787, 40.6589611)"
10,Brooklyn,64th Street,"64th Street, Brooklyn, NY",NaN,"(-74.0231939, 40.6400567)"
19,Brooklyn,Aberdeen Street,"Aberdeen Street, Brooklyn, NY",NaN,"(-73.9055714, 40.6824518)"


In [25]:
from shapely.geometry import Point
import geopandas as gpd

def make_gdf(df, point_col_name='Point'): # eg data.Point
    df['geometry'] = df[point_col_name].apply(Point)
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    return gdf

In [27]:
gdf = make_gdf(gt_geocode[gt_geocode.Point.notnull()])
gdf.head(2)

/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Boro,Address,Full_Address,NTA,Point,geometry
1,Brooklyn,1101 & 1105 Bergen Street,"1101 & 1105 Bergen Street, Brooklyn, NY",NaN,"(-73.949439375, 40.67615225)",POINT (-73.949439375 40.67615225)
5,Bronx,Carlisle Place,"Carlisle Place, Bronx, NY",NaN,"(-73.863012, 40.878598)",POINT (-73.863012 40.878598)


In [28]:
def join_nta(nta, gdf_to_join):
    gdf_to_join.crs = nta.crs
    return gpd.sjoin(gdf_to_join, nta, op='intersects')

In [50]:
joined = join_nta(nta, gdf)
joined.head(2)

,Boro,Address,Full_Address,NTA,Point,geometry,index_right,ntacode,shape_area,county_fips,ntaname,shape_leng,boro_name,boro_code
17,Brooklyn,762-764 Herkimer Place/13-21 Hunterfly,"762-764 Herkimer Place/13-21 Hunterfly, Brookl...",NaN,"(-73.927007, 40.67803)",POINT (-73.927007 40.67803),30,BK61,51619074.188,047,Crown Heights North,35635.5428561,Brooklyn,3
133,Brooklyn,"Eastern Pkwy.,W/O Franklin","Eastern Pkwy.,W/O Franklin, Brooklyn, NY",NaN,"(-73.954455, 40.670787)",POINT (-73.954455 40.670787),30,BK61,51619074.188,047,Crown Heights North,35635.5428561,Brooklyn,3


In [51]:
joined.ntaname.isnull().value_counts() # yay all were spatially joined

False    47
Name: ntaname, dtype: int64

In [52]:
joined['Latitude'] = joined.geometry.x
joined['Longitude'] = joined.geometry.y
joined = joined[['Address', 'Latitude', 'Longitude', 'ntaname']]
joined.rename(columns={'ntaname': 'NTA'}, inplace=True)
joined.head() # yay!

,Address,Latitude,Longitude,NTA
17,762-764 Herkimer Place/13-21 Hunterfly,-73.927007,40.678030,Crown Heights North
133,"Eastern Pkwy.,W/O Franklin",-73.954455,40.670787,Crown Heights North
426,Hunterfly Place and Atlantic Avenue,-73.927609,40.677592,Crown Heights North
21,1980 Lafayette Avenue/Stickball Avenue,-73.854449,40.821774,Soundview-Castle Hill-Clason Point-Harding Park
30,35th St. Bet. 35th & 36th Aves,-73.923622,40.756476,Astoria


Investigate the cases where geocoding failed

In [54]:
# now to geocode the 55 that failed earlier
failures = gt_geocode[gt_geocode.Point.isnull()]
failures.shape

(55, 5)

In [55]:
failures.to_csv('../data/greenthumb_geocode_failures.csv') # export to do manual geocoding

In [56]:
failures = pd.read_csv('../data/greenthumb_geocode_failures_cleaned.csv', index_col=0)
failures.head()

,Boro,Address,Full_Address,NTA,Point
17,Brooklyn,762-764 Herkimer Place/13-21 Hunterfly,"762-764 Herkimer Place/13-21 Hunterfly, Brookl...",NaN,"40.678030, -73.927007"
21,Bronx,1980 Lafayette Avenue/Stickball Avenue,"1980 Lafayette Avenue/Stickball Avenue, Bronx, NY",NaN,"40.821774, -73.854449"
30,Queens,35th St. Bet. 35th & 36th Aves,"35th St. Bet. 35th & 36th Aves, Queens, NY",NaN,"40.756476, -73.923622"
41,Brooklyn,93-95 Malcolm X Avenue,"93-95 Malcolm X Avenue, Brooklyn, NY",NaN,"40.690746, -73.929372"
101,Manhattan,320 96th Street NY NY 10128,320 96th Street NY NY 10128,NaN,"40.783460, -73.945095"


In [57]:
def split_pt(s):
    temp = s.split(', ')
    return tuple([float(temp[1]), float(temp[0])]) # long, lat

failures['Point'] = failures.Point.apply(split_pt)
failures.head()

,Boro,Address,Full_Address,NTA,Point
17,Brooklyn,762-764 Herkimer Place/13-21 Hunterfly,"762-764 Herkimer Place/13-21 Hunterfly, Brookl...",NaN,"(-73.927007, 40.67803)"
21,Bronx,1980 Lafayette Avenue/Stickball Avenue,"1980 Lafayette Avenue/Stickball Avenue, Bronx, NY",NaN,"(-73.854449, 40.821774)"
30,Queens,35th St. Bet. 35th & 36th Aves,"35th St. Bet. 35th & 36th Aves, Queens, NY",NaN,"(-73.923622, 40.756476)"
41,Brooklyn,93-95 Malcolm X Avenue,"93-95 Malcolm X Avenue, Brooklyn, NY",NaN,"(-73.929372, 40.690746)"
101,Manhattan,320 96th Street NY NY 10128,320 96th Street NY NY 10128,NaN,"(-73.945095, 40.78346)"


In [58]:
gdf = make_gdf(failures, 'Point')
gdf.head(2)

,Boro,Address,Full_Address,NTA,Point,geometry
17,Brooklyn,762-764 Herkimer Place/13-21 Hunterfly,"762-764 Herkimer Place/13-21 Hunterfly, Brookl...",NaN,"(-73.927007, 40.67803)",POINT (-73.927007 40.67803)
21,Bronx,1980 Lafayette Avenue/Stickball Avenue,"1980 Lafayette Avenue/Stickball Avenue, Bronx, NY",NaN,"(-73.854449, 40.821774)",POINT (-73.854449 40.821774)


In [62]:
joined2 = join_nta(nta, gdf)
joined2.head(2)

,Boro,Address,Full_Address,NTA,Point,geometry,index_right,ntacode,shape_area,county_fips,ntaname,shape_leng,boro_name,boro_code
17,Brooklyn,762-764 Herkimer Place/13-21 Hunterfly,"762-764 Herkimer Place/13-21 Hunterfly, Brookl...",NaN,"(-73.927007, 40.67803)",POINT (-73.927007 40.67803),30,BK61,51619074.188,047,Crown Heights North,35635.5428561,Brooklyn,3
133,Brooklyn,"Eastern Pkwy.,W/O Franklin","Eastern Pkwy.,W/O Franklin, Brooklyn, NY",NaN,"(-73.954455, 40.670787)",POINT (-73.954455 40.670787),30,BK61,51619074.188,047,Crown Heights North,35635.5428561,Brooklyn,3


In [63]:
joined2['Latitude'] = joined2.geometry.x
joined2['Longitude'] = joined2.geometry.y
joined2 = joined2[['Address', 'Latitude', 'Longitude', 'ntaname']]
joined2.rename(columns={'ntaname': 'NTA'}, inplace=True)
joined2.head()

,Address,Latitude,Longitude,NTA
17,762-764 Herkimer Place/13-21 Hunterfly,-73.927007,40.678030,Crown Heights North
133,"Eastern Pkwy.,W/O Franklin",-73.954455,40.670787,Crown Heights North
426,Hunterfly Place and Atlantic Avenue,-73.927609,40.677592,Crown Heights North
21,1980 Lafayette Avenue/Stickball Avenue,-73.854449,40.821774,Soundview-Castle Hill-Clason Point-Harding Park
30,35th St. Bet. 35th & 36th Aves,-73.923622,40.756476,Astoria


# Data cleaning 5: combining all datasets


In [67]:
COLS = ['Address', 'Latitude', 'Longitude', 'NTA'] # columns i want in the final dataset

In [68]:
# dataset 1
nyrp = pd.read_csv('../data/NYRP_NTA.csv', index_col=0)
nyrp.head(2)

,address,coords,lat,long,borough,geometry,index_right,ntacode,shape_area,county_fips,ntaname,shape_leng,boro_name,boro_code
0,735 East 211th St,"40.877499,-73.863489",40.877499,-73.863489,Bronx,POINT (-73.863489 40.877499),75,BX44,3.627360e+07,5,Williamsbridge-Olinville,27351.077379,Bronx,2
1,1818 Bathgate Ave,"40.845051,-73.897747",40.845051,-73.897747,Bronx,POINT (-73.897747 40.845051),128,BX01,1.645162e+07,5,Claremont-Bathgate,29972.777720,Bronx,2


In [71]:
# rename columns for concat
nyrp.rename(columns={
    'address': 'Address',
    'lat': 'Latitude',
    'long': 'Longitude',
    'borough': 'Borough',
    'ntaname': 'NTA',
}, inplace=True) 
nyrp = nyrp[COLS]
nyrp.head(2)

,Address,Latitude,Longitude,NTA
0,735 East 211th St,40.877499,-73.863489,Williamsbridge-Olinville
1,1818 Bathgate Ave,40.845051,-73.897747,Claremont-Bathgate


In [74]:
greenthumb = greenthumb[greenthumb.NTA.notnull()][COLS] # dataset 2
greenthumb.head(2)

,Address,Latitude,Longitude,NTA
0,626 East 11th Street,40.727124,-73.978677,Lower East Side
2,1651 Madison Avenue,40.796295,-73.947533,East Harlem South


In [76]:
greenthumb_geocoded = joined # dataset 3, gardens in greenthumb that didn't have NTA info initially, which i later geocoded
greenthumb_geocoded.head(2)

,Address,Latitude,Longitude,NTA
17,762-764 Herkimer Place/13-21 Hunterfly,-73.927007,40.678030,Crown Heights North
133,"Eastern Pkwy.,W/O Franklin",-73.954455,40.670787,Crown Heights North


In [77]:
greenthumb_geocoded2 = joined2
greenthumb_geocoded2.head(2)

,Address,Latitude,Longitude,NTA
17,762-764 Herkimer Place/13-21 Hunterfly,-73.927007,40.678030,Crown Heights North
133,"Eastern Pkwy.,W/O Franklin",-73.954455,40.670787,Crown Heights North


In [79]:
gardens_combined = pd.concat([nyrp, greenthumb, greenthumb_geocoded, greenthumb_geocoded2], ignore_index=True)
gardens_combined.head()

,Address,Latitude,Longitude,NTA
0,735 East 211th St,40.877499,-73.863489,Williamsbridge-Olinville
1,1818 Bathgate Ave,40.845051,-73.897747,Claremont-Bathgate
2,1017 Teller Ave,40.828150,-73.914356,East Concourse-Concourse Village
3,1328 Clay Ave,40.834934,-73.908892,East Concourse-Concourse Village
4,2044 Prospect Ave,40.845818,-73.887851,East Tremont


In [80]:
gardens_combined.shape

(581, 4)

# Data cleaning: remove duplicates ie gardens of same address

In [93]:
# num of duplicates
len(gardens_combined[gardens_combined.duplicated(subset='Address')])

54

In [95]:
gardens_combined.shape # num gardens before

(581, 4)

In [96]:
gardens_combined.drop_duplicates(subset='Address', keep='first', inplace=True)
gardens_combined.shape # num gardens after

(527, 4)

In [97]:
# save dataset
gardens_combined.to_csv('../data/NYC_Community_Gardens_combined.csv')